In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [6]:
# Load dataset
df = pd.read_csv('vehicles.csv')

## Data Preprocessing and Cleaning

In [7]:
df.shape

(426880, 26)

In [8]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [10]:
df.isnull().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

#### Drop Redundant Columns

In [11]:
cols_to_drop = [
    'id', 'url', 'region_url', 'image_url', 'description', 
    'county', 'VIN', 'posting_date',
]

df = df.drop(columns=cols_to_drop)

#### Drop column `size` due to high missingness (~72%)

In [12]:
df = df.drop('size',axis =1)

#### Filter by year (keeping entries after 1990)

In [13]:
unique_years = df["year"].unique()
print("Unique years:", unique_years)

Unique years: [  nan 2014. 2010. 2020. 2017. 2013. 2012. 2016. 2019. 2011. 1992. 2018.
 2004. 2015. 2001. 2006. 1968. 2003. 2008. 2007. 2005. 1966. 2009. 1998.
 2002. 1999. 2021. 1997. 1976. 1969. 1995. 1978. 1954. 1979. 1970. 1974.
 1996. 1987. 2000. 1955. 1960. 1991. 1972. 1988. 1994. 1929. 1984. 1986.
 1989. 1973. 1946. 1933. 1958. 1937. 1985. 1957. 1953. 1942. 1963. 1977.
 1993. 1903. 1990. 1965. 1982. 1948. 1983. 1936. 1932. 1951. 1931. 1980.
 1967. 1971. 1947. 1981. 1926. 1962. 1975. 1964. 1934. 1952. 1940. 1959.
 1950. 1930. 1956. 1922. 1928. 2022. 1901. 1941. 1924. 1927. 1939. 1923.
 1949. 1961. 1935. 1918. 1900. 1938. 1913. 1916. 1943. 1925. 1921. 1915.
 1945. 1902. 1905. 1920. 1944. 1910. 1909.]


In [14]:
years_before_1990 = df[df['year'] < 1990]

In [15]:
years_before_1990.shape

(12383, 17)

In [16]:
df = df[df['year'] >= 1990]

#### Remove outliers in `price` and `odometer`

In [17]:
df = df[(df['price'] >= 100) & (df['price'] <= 200000)]

In [18]:
df = df[df['odometer'] <= 300000]

#### Imputing Missing Values in the `fuel` Column
**Imputation Logic:**
Since the null values represent a very small proportion of the data (~0.5%), and `'gas'` is the dominant value by a large margin, it is reasonable to fill the missing values with the mode (`'gas'`).

In [19]:
df["fuel"].value_counts()

fuel
gas         312419
other        27577
diesel       24378
hybrid        4830
electric      1598
Name: count, dtype: int64

In [20]:
fuel_mode = df['fuel'].mode()[0]
df['fuel'] = df['fuel'].fillna(fuel_mode)

In [21]:
df.shape

(372910, 17)

#### Removing Duplicate Rows

In [22]:
num_duplicates = df.duplicated().sum()
print(f"Number of exact duplicate rows: {num_duplicates}")

Number of exact duplicate rows: 45783


In [23]:
df = df.drop_duplicates()

In [24]:
df.shape

(327127, 17)

In [25]:
df.isna().sum()

region               0
price                0
year                 0
manufacturer     10382
model             3031
condition       117759
cylinders       131566
fuel                 0
odometer             0
title_status      5799
transmission      1270
drive            98294
type             67010
paint_color      92394
state                0
lat               3254
long              3254
dtype: int64

#### Handling of High-Missing Categorical Columns

*Impute missing values with 'Unknown'*

In [26]:
# List of high-missing categorical columns to impute with 'Unknown'
cols_to_impute_unknown = ['condition', 'cylinders', 'drive', 'paint_color', 'type']

# Impute missing values with 'Unknown'
for col in cols_to_impute_unknown:
    df[col] = df[col].fillna('Unknown')

In [27]:
missing_values = df.isna().sum()
missing_columns = missing_values[missing_values > 0]
print("Columns with missing values:")
missing_columns

Columns with missing values:


manufacturer    10382
model            3031
title_status     5799
transmission     1270
lat              3254
long             3254
dtype: int64

#### Imputing Missing `lat` and `long` Values Based on `state` coordinates

In [28]:
# Calculate mean 'lat' and 'long' grouped by 'state'
state_means = df.groupby('state')[['lat', 'long']].mean()

# Identify rows where both 'lat' and 'long' are missing
missing_both = df['lat'].isnull() & df['long'].isnull()

# Impute missing 'lat' and 'long' using the calculated means
df.loc[missing_both, 'lat'] = df.loc[missing_both, 'state'].map(state_means['lat'])
df.loc[missing_both, 'long'] = df.loc[missing_both, 'state'].map(state_means['long'])

# Verify imputation
df[['lat', 'long']].isnull().sum()

lat     0
long    0
dtype: int64

#### Handling Missing Values: `title_status` and `transmission`
**Overview of Columns**

In [29]:
df['title_status'].value_counts()

title_status
clean         309822
rebuilt         6403
salvage         3363
lien            1347
missing          280
parts only       113
Name: count, dtype: int64

In [30]:
df["transmission"].value_counts()

transmission
automatic    250073
other         59573
manual        16211
Name: count, dtype: int64

**Given the low percentage of missing values, mode imputation is chosen for both columns.**

**Mode Values:**
   - **`title_status`:** The mode is `"clean"`, which accounts for over 90% of the values.
   - **`transmission`:** The mode is `"automatic"`, the dominant category.

In [31]:
title_status_mode = df['title_status'].mode()[0]
df['title_status'] = df['title_status'].fillna(title_status_mode)

transmission_mode = df['transmission'].mode()[0]
df['transmission'] = df['transmission'].fillna(transmission_mode)

In [32]:
missing_values = df.isna().sum()
missing_columns = missing_values[missing_values > 0]
print("Columns with missing values:")
missing_columns

Columns with missing values:


manufacturer    10382
model            3031
dtype: int64

#### Predictive Imputation for `manufacturer`

**Split into known and missing subsets**

In [33]:
df_manuf_known = df[df['manufacturer'].notnull()].copy()
df_manuf_missing = df[df['manufacturer'].isnull()].copy()

**Define Features for Prediction**

In [34]:
features_manuf = [
    col for col in df.columns 
    if col not in ('manufacturer', 'model')
]

**Label-encode the features**

In [35]:
# Initialize a dictionary to store encoders for each feature
encoders_manuf = {}

for col in features_manuf:
    enc = LabelEncoder()
    
    # Combine known and missing data for the column
    combined_series = pd.concat([df_manuf_known[col].astype(str), 
                                 df_manuf_missing[col].astype(str)], 
                                ignore_index=True)
    
    # Fit the encoder on the combined data
    enc.fit(combined_series)
    
    # Transform the feature columns
    df_manuf_known[col] = enc.transform(df_manuf_known[col].astype(str))
    df_manuf_missing[col] = enc.transform(df_manuf_missing[col].astype(str))
    
    # Store the encoder
    encoders_manuf[col] = enc

# Encode the target variable ('manufacturer')
manuf_encoder = LabelEncoder()
df_manuf_known['manufacturer_encoded'] = manuf_encoder.fit_transform(df_manuf_known['manufacturer'].astype(str))

**Train the Random Forest Classifier**

In [36]:
# Define training data
X_train_manuf = df_manuf_known[features_manuf]
y_train_manuf = df_manuf_known['manufacturer_encoded']

# Initialize and train the classifier
rf_manuf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_manuf.fit(X_train_manuf, y_train_manuf)


RandomForestClassifier(n_jobs=-1, random_state=42)

**Predict the missing `manufacturer`**

In [37]:
# Define the data to predict
X_pred_manuf = df_manuf_missing[features_manuf]

# Predict using the trained classifier
y_manuf_pred = rf_manuf.predict(X_pred_manuf)

# Convert numeric predictions back to original manufacturer labels
manuf_pred_labels = manuf_encoder.inverse_transform(y_manuf_pred)

# Assign the predicted labels back to the original DataFrame
df.loc[df_manuf_missing.index, 'manufacturer'] = manuf_pred_labels

print("Predictive imputation for 'manufacturer' completed.")

Predictive imputation for 'manufacturer' completed.


#### Impute Missing `model` Values with "Unknown" 

Given the **high computational cost** of predictive imputation and the fact that the missing values in the `model` column were **very few (~1%)**, I opted for a simple imputation by filling the missing values with `"Unknown"`. 

In [42]:
df['model'] = df['model'].fillna('Unknown')

In [44]:
df.isna().sum()

region          0
price           0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
type            0
paint_color     0
state           0
lat             0
long            0
dtype: int64

#### Save the updated DataFrame

In [45]:
df.to_csv('preprocessed_used_cars_final.csv', index=False)